In [ ]:
import requests
import json

def get_data():

    url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001"
    params = {
        "Authorization": "CWB-8C566AAC-E3C9-48AE-9A42-4563DC4452AC", # 請改成自己的 API 授權碼
        "locationName": "臺中市",
    }

    response = requests.get(url, params=params)
    # print(response.status_code)

    if response.status_code == 200:
        # print(response.text)
        data = json.loads(response.text)

        location = data["records"]["location"][0]["locationName"]
        print(location)

        weather_elements = data["records"]["location"][0]["weatherElement"]
        for i in range(len(weather_elements[0])+1):

          start_time = weather_elements[0]["time"][i]["startTime"]
          end_time = weather_elements[0]["time"][i]["endTime"]
          weather_state = weather_elements[0]["time"][i]["parameter"]["parameterName"]
          rain_prob = weather_elements[1]["time"][i]["parameter"]["parameterName"]
          min_tem = weather_elements[2]["time"][i]["parameter"]["parameterName"]
          comfort = weather_elements[3]["time"][i]["parameter"]["parameterName"]
          max_tem = weather_elements[4]["time"][i]["parameter"]["parameterName"]

          print(start_time)
          print(end_time)
          print(weather_state)
          print(rain_prob)
          print(min_tem)
          print(comfort)
          print(max_tem)
          print()

    else:
        print("Can't get data!")

get_data()

臺中市
2023-04-17 00:00:00
2023-04-17 06:00:00
晴時多雲
0
22
舒適
24

2023-04-17 06:00:00
2023-04-17 18:00:00
晴時多雲
0
22
舒適至悶熱
32

2023-04-17 18:00:00
2023-04-18 06:00:00
晴時多雲
0
22
舒適
28



台中現在天氣溫溼度<br>
利用 google sheet 當 database<br>
https://ithelp.ithome.com.tw/articles/10241204<br>
https://medium.com/sheiun/google-spreadsheet-%E5%BF%AB%E9%80%9F%E6%90%AD%E5%BB%BA%E5%85%8D%E8%B2%BB%E8%B3%87%E6%96%99%E5%BA%AB-%E4%B8%8B-4b4900baa5f0<br>

In [ ]:
# 安裝套件指定版本
!pip install streamlit
!pip install python-highcharts==0.4.2 > log.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h2> 畫出溫溼度 highcharts </h2>
[股市](https://colab.research.google.com/drive/1W1kH3cwNUTj7hMMyF8W4wcehiWLSyAUF?usp=sharing&fbclid=IwAR238dbwrknyn43NeAwED1vX2Gf_tjE4mwuRpbGCT4AuEMN8LinO3Ef6mDk)

[Python教學系列-視覺化套件 Python-highcharts](https://www.facebook.com/groups/pythontw/posts/10162690900473438/?locale=pt_BR)

[highcharts](https://www.highcharts.com/demo/column-basic)

In [ ]:
import requests
import json
import streamlit as st
from highcharts import Highchart
import datetime
from IPython.display import HTML,display
import os
import sqlite3
import random

def get_wether():
  url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001"
  params = {
      "Authorization": "CWB-8C566AAC-E3C9-48AE-9A42-4563DC4452AC", # 請改成自己的 API 授權碼
      "stationId": "C0F9P0",
      "elementName": "TEMP,HUMD,Weather",
  }
  response = requests.get(url, params=params)

  data = json.loads(response.text)
  temp=data["records"]["location"][0]["weatherElement"][0]["elementValue"]
  hum=data["records"]["location"][0]["weatherElement"][1]["elementValue"]
  hum=float(hum)*100
  weather=data["records"]["location"][0]["weatherElement"][2]["elementValue"]
  print(data["records"]["location"][0]["weatherElement"])

  print(f'溫度: {temp} , 濕度: {hum} %, 天氣: {weather}')
  return temp,hum

def insertsql(temp,hum):
  # Connect to SQLite database
  conn = sqlite3.connect('weather.db')
  c = conn.cursor()

  # # Create table if it doesn't exist
  # c.execute('''CREATE TABLE IF NOT EXISTS weather
  #             (id INTEGER PRIMARY KEY, temperature TEXT, humidity TEXT)''')
  c.execute('''CREATE TABLE IF NOT EXISTS weather
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
             temperature TEXT,
             humidity TEXT);''')
              
  # Use Streamlit to display and save data
  st.title('Taichung Temperature and Humidity')
  st.write(f'Temperature: {temp} ℃')
  st.write(f'Humidity: {hum} %')
  c.execute(f"INSERT INTO weather (temperature, humidity) VALUES ('{temp}', '{hum}')")
  conn.commit()

  # Close connection
  conn.close()

def selectAll():
  # 連接到SQLite3資料庫
  conn = sqlite3.connect('weather.db')

  # 執行SELECT查詢，選取所有資料
  cursor = conn.execute("SELECT * FROM weather")

  # 獲取查詢結果的所有列
  rows = cursor.fetchall()

  time_list=[]
  temp_list=[]
  hum_list=[]
  # 顯示結果
  for row in rows:
    time_list.append(row[1])
    temp_list.append(float(row[2]))
    hum_list.append(float(row[3]))
    # print(row)

  # 關閉資料庫連接
  conn.close()

  return time_list,temp_list,hum_list

def drawHighchart(time_list,temp_list,hum_list):

  # 客制化調整參數
  color = '#4285f4' # 線的顏色 (red/green/blue/purple)
  linewidth = 2 # 線的粗細
  title = "temperature & humidity in Taichung" # 標題名稱
  width = 800 # 圖的寬度
  height = 500 # 圖的高度

  # 繪圖設定
  H = Highchart(width=width,height=height)
  temp_line= [[index,s] for index,s in zip(time_list,temp_list)]
  hum_line = [[index,s] for index,s in zip(time_list,hum_list)]
  print(temp_line)
  # print(hum_line)
  H.add_data_set(temp_line,'line','temperature',color='blue')
  H.add_data_set(hum_line,'line','humidity',color='green')

  H.set_options('xAxis',{'type':'datetime'})
  H.set_options('title',{'text':title,'style':{'color':'black'}}) # 設定title
  H.set_options('plotOptions',{'line':{'lineWidth':linewidth,'dataLabels':{'enabled': False}}}) # 設定線的粗度
  H.set_options('tooltip',{'shared':True,'crosshairs':True}) # 設定為可互動式

  # 顯示圖表
  H.save_file('chart')
  display(HTML('chart.html'))
  os.remove('chart.html')

def randInsert10():
  # 連接到SQLite3資料庫（如果不存在就建立）
  conn = sqlite3.connect('weather.db')
  c = conn.cursor()

  # 建立資料表
  # Create table if it doesn't exist
  c.execute('''CREATE TABLE IF NOT EXISTS weather
              (id INTEGER PRIMARY KEY AUTOINCREMENT,
              timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
              temperature TEXT,
              humidity TEXT);''')

  # 插入10個隨機的溫度和濕度數值
  for i in range(10):
      temp = round(random.uniform(20, 30), 2)
      hum = round(random.uniform(40, 100), 2)
        # Use Streamlit to display and save data
      st.title('Taichung Temperature and Humidity')
      st.write(f'Temperature: {temp} ℃')
      st.write(f'Humidity: {hum} %')
      c.execute(f"INSERT INTO weather (temperature, humidity) VALUES ('{temp}', '{hum}')")
      conn.commit()

  # 關閉資料庫連接
  conn.close()


# 取得溫度
temp,hum = get_wether()
# 將溫溼度存入資料庫
insertsql(temp,hum)
# 抓所有溫濕度
time_list,temp_list,hum_list = selectAll()
# print(temp_list,hum_list)
# 隨機新增10筆資料
randInsert10()
# 畫出 highchart
drawHighchart(time_list,temp_list,hum_list)

[{'elementName': 'TEMP', 'elementValue': '24.0'}, {'elementName': 'HUMD', 'elementValue': '0.76'}, {'elementName': 'Weather', 'elementValue': '晴'}]
溫度: 24.0 , 濕度: 76.0 %, 天氣: 晴
[['2023-04-16 17:54:06', 24.0], ['2023-04-16 17:54:06', 28.05], ['2023-04-16 17:54:06', 21.74], ['2023-04-16 17:54:06', 26.48], ['2023-04-16 17:54:06', 20.7], ['2023-04-16 17:54:06', 21.51], ['2023-04-16 17:54:06', 20.51], ['2023-04-16 17:54:06', 24.81], ['2023-04-16 17:54:06', 23.9], ['2023-04-16 17:54:06', 20.22], ['2023-04-16 17:54:06', 23.89], ['2023-04-16 17:54:26', 24.0]]


水庫資料<br>
[STAEM 教學](https://steam.oxxostudio.tw/category/python/spider/beautiful-soup.html)<br>
[用數據看台灣](https://water.taiwanstat.com/)<br>

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://water.taiwanstat.com/'
web = requests.get(url)
soup = BeautifulSoup(web.text, "html.parser")
reservoir = soup.select('.reservoir')     # 取得所有 class 為 reservoir 的 tag

for i in reservoir:
  print(i.find('div', class_='name').get_text(), end=' ')  # 取得內容的 class 為 name 的 div 文字
  print(i.find('h5').get_text(), end=' ')   # 取得內容 h5 tag 的文字
  print()

新山水庫(基隆) 7001.1萬立方公尺 
翡翠水庫(台北、新北) 26722.51萬立方公尺 
石門水庫(新北、桃園、新竹) 4025.63萬立方公尺 
永和山水庫(新竹、苗栗) 685.598萬立方公尺 
寶山水庫(新竹) 7001.1萬立方公尺 
寶山第二水庫(新竹) 7001.1萬立方公尺 
明德水庫(苗栗) 262.0316萬立方公尺 
鯉魚潭水庫(苗栗、台中) 9857.34萬立方公尺 
德基水庫(台中) 762.27萬立方公尺 
石岡壩(台中) 11486萬立方公尺 
日月潭水庫(南投) 5741萬立方公尺 
霧社水庫(南投) 3269.4萬立方公尺 
湖山水庫(雲林、彰化、嘉義) 820.884613萬立方公尺 
仁義潭水庫(嘉義) 820.884613萬立方公尺 
蘭潭水庫(嘉義) 1086.97萬立方公尺 
白河水庫(台南) 1086.97萬立方公尺 
曾文水庫(嘉義、台南) 1086.97萬立方公尺 
烏山頭水庫(台南) 1086.97萬立方公尺 
南化水庫(台南、高雄) 1086.97萬立方公尺 
阿公店水庫(高雄) 1086.97萬立方公尺 
牡丹水庫(屏東) 1086.97萬立方公尺 
